In [47]:
import os
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain_experimental.pal_chain import PALChain
from langchain.chains import SimpleSequentialChain,SequentialChain
from langchain.memory import SimpleMemory

In [2]:
prompt=PromptTemplate(
    input_variables=['product'],
    template="What is a good name for a company that makes {product}?",
)

print(prompt.format(product="podcast player"))

What is a good name for a company that makes podcast player?


In [3]:
llm=OpenAI(
    model_name="text-davinci-003",
    temperature=0.9)
llmchain=LLMChain(llm=llm,prompt=prompt)
llmchain.run("podcast player")

'\n\nPodJogger.'

In [4]:
chatopenai=ChatOpenAI(
    model="gpt-3.5-turbo")
llmchain_chat=LLMChain(llm=chatopenai,prompt=prompt)
llmchain_chat.run("podcast player")

'PodcastPro'

In [5]:
palchain=PALChain.from_math_prompt(llm=llm,verbose=True)
palchain.run("if my age is half of my dad's age and he is going to be 60 next year,what is my current age?")



> Entering new PALChain chain...


Python REPL can execute arbitrary code. Use with caution.


def solution():
    """if my age is half of my dad's age and he is going to be 60 next year,what is my current age?"""
    dad_age_now = 59
    my_age_now = dad_age_now / 2
    result = my_age_now
    return result

> Finished chain.


'29.5'

TypeError: Chain.prep_inputs() missing 1 required positional argument: 'inputs'

In [28]:
llm=OpenAI(temperature=0.9)
tools=load_tools(["llm-math"],llm=llm)  #used llm-math instead of pal-math. it seems like pal-math is dropped from avaialble tools in langchain tools

agent=initialize_agent(tools,
                       llm,
                       agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                       verbose=True)

agent.run("if my age is half of my dad's age and he is going to be 60 next year,what is my current age?")



> Entering new AgentExecutor chain...
 I need to know the age difference between my dad and I
Action: Calculator
Action Input: 60 - half of 60

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: Request failed due to server shutdown {
  "error": {
    "message": "Request failed due to server shutdown",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'Request failed due to server shutdown', 'type': 'server_error', 'param': None, 'code': None}} {'Date': 'Tue, 26 Sep 2023 16:52:02 GMT', 'Content-Type': 'application/json', 'Content-Length': '141', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'openai-model': 'text-davinci-003', 'openai-organization': 'user-ockh9u2vxdxb3rveeylzyzgb', 'openai-processing-ms': '10419', 'openai-version': '2020-10-01', 'strict-transport-security': 'max-age=15724800; includeSubDomains', 'x-ratelimit-limit-requests': '3000', 'x-ratelimit-limit-tokens': '250000', 'x-ratelimit-limit-tokens_usage_based': '250000', 'x-ratelimit-remaining-requests': '2999', 'x-ratelimit-rem


Observation: Answer: 30.0
Thought: I now know my dad's age and half of his age
Action: Calculator
Action Input: 30 / 2
Observation: Answer: 15.0
Thought: I now know my current age
Final Answer: 15

> Finished chain.


'15'

In [24]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [25]:
agent.run("My age is half of my dad's age. Next year he is going to be same age as Demi Moore. What is my current age?")



> Entering new AgentExecutor chain...
 Calculating my dad's age will tell me my age.
Action: Calculator
Action Input: Demi Moore's age (57) + 1
Observation: Answer: 58
Thought: I should divide by 2 to figure out my age
Action: Calculator 
Action Input: 58/2
Observation: Answer: 29.0
Thought: I now know the final answer
Final Answer: My current age is 29.

> Finished chain.


'My current age is 29.'

In [30]:
#Chain1 - solve math problem, get the age
chainone=agent

#Chain2 - suggest age-appropriate gift
template="""You are a gift recomender. Giev a person's age,\n
it is your job to suggest an appropriate gift for them.

Person Age:
{age}
Suggest gift:"""

prompt_template=PromptTemplate(input_variables=["age"],template=template)
chaintwo=LLMChain(llm=llm,prompt=prompt_template)

In [32]:
overall_chain=SimpleSequentialChain(
    chains=[chainone,chaintwo],
    verbose=True
)

In [33]:
question="if my age is half of my dad's age and he is going to be 60 next year,what is my current age?"
overall_chain.run(question)



> Entering new SimpleSequentialChain chain...


> Entering new AgentExecutor chain...
 I know my dad's age is going to be 60 next year, which means his current age is 59.
Thought: If my age is half of his current age, then my age must be half of 59.
Action: Calculator
Action Input: 59/2
Observation: Answer: 29.5
Thought: I now know the final answer
Final Answer: My current age is 29.5 years old.

> Finished chain.
My current age is 29.5 years old.
 
A great gift idea for someone of this age would be a gift card for a new experience such as an adventure outing, a cooking class, a tasting event, or a subscription box service. This will give them the opportunity to explore and try something new, creating memories that will last a lifetime.

> Finished chain.


' \nA great gift idea for someone of this age would be a gift card for a new experience such as an adventure outing, a cooking class, a tasting event, or a subscription box service. This will give them the opportunity to explore and try something new, creating memories that will last a lifetime.'

In [36]:
template = """You are a gift recommender. Given a person's age,\n
 it is your job to suggest an appropriate gift for them. If age is under 10,\n
 the gift should cost no more than {budget} otherwise it should cost atleast 10 times {budget}.

 Person Age:
 {output}
 Suggest gift:"""
prompt_template=PromptTemplate(input_variables=["output",'budget'],template=template)
chaintwo=LLMChain(llm=llm,prompt=prompt_template)

In [37]:
print(agent.agent.llm_chain.output_keys)

['text']


In [48]:
overall_chain=SequentialChain(
    input_variables=["input"],
    memory=SimpleMemory(memories={"budget":"100 GBP"}),
    chains=[agent,chaintwo],
    verbose=True
)

Passing input variables to the SequentialChain is mandatory unlike SimpleSequentialChain. it is the list of input variables expected by the first entity in the chain. Here it our agent.
We can find the name of the variable used in the input prompt that the agent is using

In [50]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


At the end we can see that the questions being asked are stored in the variable input

In [51]:
#the variable name can also be found like this
print(agent.agent.llm_chain.prompt.input_variables)

['input', 'agent_scratchpad']


SimpleMEmory is an easy way of storing context and other information that shouldnt change between prompts. It requires one parameter at the time of initialization - memories. you can pass elements to it dict form

In [52]:
question="if my age is half of my dad's age and he is going to be 60 next year,what is my current age?"
overall_chain.run(question)



> Entering new SequentialChain chain...


> Entering new AgentExecutor chain...
 I need to solve for x.
Action: Calculator
Action Input: .5x+1=60
Observation: Answer: 29.5
Thought: I now know the final answer
Final Answer: My current age is 29.5.

> Finished chain.

> Finished chain.


'\nGiven your age, I would suggest a gift that is at least £1000. Some potential ideas could be a set of designer luggage, a smartwatch, a digital camera, a gaming console, a smart home device, or a luxury spa treatment.'